In [3]:
import pandas as pd

import openai

In [4]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [5]:
def query_openai(model, prompt, max_tokens, temperature):
    response = openai.Completion.create(
        model=model,
        prompt=prompt,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
    )

    return response['choices'][0]['text']

query_openai('text-davinci-003', 'Adobe, empowering the world through', 3, 0.7)

' digital experiences\n'

In [10]:
df_candidates = pd.read_csv('candidates.csv', index_col=0)

df_candidates.head(20)

,weather,pivot,candidates
0,warm,"Tripoli, Libya","Tunis, Tunisia; Marrakech, Morocco; Algiers, A..."
1,warm,"Vientiane, Laos","Hanoi, Vietnam; Siem Reap, Cambodia; Bangkok, ..."
2,warm,"Suva, Fiji","Apia, Samoa; Port Vila, Vanuatu; Honiara, Solo..."
3,warm,"La Paz, Mexico","Cancun, Mexico; Cabo San Lucas, Mexico; Mazatl..."
4,warm,"Garissa, Kenya","Mombasa, Kenya; Lamu, Kenya; Wajir, Kenya; Kis..."
5,warm,"Ho Chi Minh City, Vietnam","Hanoi, Vietnam; Bangkok, Thailand; Siem Reap, ..."
6,warm,"Brasília, Brazil","Quito, Ecuador; San Jose, Costa Rica; La Paz, ..."
7,warm,"Makassar, Indonesia","Bandar Seri Begawan, Brunei ; Kota Kinabalu, M..."
8,warm,"Brisbane, Australia","Cairns, Australia ; Townsville, Australia ; Da..."
9,warm,"Hat Yai, Thailand","Phuket, Thailand; Kota Kinabalu, Malaysia; Sin..."


### Focus on a single example from the loaded data

In [13]:
example = df_candidates.iloc[6]

example

weather                                                    warm
pivot                                          Brasília, Brazil
candidates    Quito, Ecuador; San Jose, Costa Rica; La Paz, ...
Name: 6, dtype: object

In [73]:
all_candidates = [example['pivot']] + example['candidates'].split('; ')

n_candidates = len(all_candidates)

### Generate touristic highlights for each city on the candidates list

In [61]:
max_highlight_len = 20

prompt = "For each one of the cities below, write its touristic highlights in less than 20 words, separating them with a line break." + '\n'.join(all_candidates)

highlights = '\n'.join([query_openai('text-davinci-003', prompt, n_candidates*max_highlight_len, 0.7) for _ in range(3)])

In [62]:
highlights

"\n\nBrasília: Monumental Axis, Cathedral of Brasilia, Paranoá Lake\nQuito: Historic Center, Equator Monument, Mitad del Mundo\nSan Jose: National Theater, Gold Museum, La Paz Waterfall\nLa Paz: Witches' Market, Tiwanaku Ruins, Moon Valley\nAsuncion: Government Palace, National Pantheon, Museums\nCaracas: Avila National Park, Caracas Cathedral, Los Caobos Park\nBogota: Gold Museum, Monserrate Hill, Botero Museum\nLima: Historic Center, Miraflores, Larco Museum\nSantiago: Plaza de Armas, San Cristobal Hill, La Chascona\nBuenos Aires: La Boca, Teatro Colon, Recoleta Cemetery\nManagua: Old Cathedral, Masaya Volcano, Tiscapa Lagoon\nMontevideo: Ciudad Vieja, Palacio Salvo, Rambla\nGuatemala City: National Palace, La Merced Church, Tikal\nSan Salvador: Monument to Heroes, Metropolitan Cathedral, Cinquera Forest\nPanama City: Casco Viejo, Panama Canal, Amador Causeway\nMexico City: Historic Center, Chapultepec Park, Palacio de Bellas Artes\nKingston: Bob Marley Museum, Hope Botanic Garden, D

In [63]:
prompt = f"Consolidate the lists below into a single, exhaustive list of highlights for each city.\n{highlights}\nNew list:"

highlights = query_openai('text-davinci-003', prompt, n_candidates*max_highlight_len*2, 0.7)

In [64]:
highlights

"\nBrasília: Monumental Axis, Brasília Cathedral, Paranoá Lake, Juscelino Kubitschek Memorial\nQuito: Historic Centre, Basilica del Voto Nacional, La Mitad del Mundo, Equator Monument\nSan Jose: National Theatre, Central Market, Poás Volcano National Park, Gold Museum, La Paz Waterfall\nLa Paz: Valle de la Luna, Mercado de las Brujas, Tiwanaku, Witches' Market\nAsuncion: Asuncion Bay, Museo del Barro, Palacio de los López, Government Palace, National Pantheon, Museums\nCaracas: Avila National Park, El Hatillo, Museo de Arte Contemporáneo, Avila Cable Car, Los Caobos Park\nBogota: Gold Museum, Monserrate Hill, La Candelaria, Botero Museum\nLima: Historic Centre, Plaza de Armas, Huaca Pucllana, Miraflores, Larco Museum\nSantiago: Cerro Santa Lucia, Plaza de Arma, Museo Chileno de Arte Precolombino, San Cristobal Hill, La Chascona\nBuenos Aires: La Boca, Obelisco, Recoleta Cemetery, Teatro Colon, Café Tortoni\nManagua: Parque Nacional, Masaya Volcano, Monumento Nacional, Old Cathedral, Ti

### Use the touristic highlights to generate potential clarification questions

In [76]:
prompt = f"Use the characteristics that are common across cities to formulate questions that can be used to shorten the list of cities below. Separate them with a line break.\nCity characteristics:{highlights}\nQuestions:"

questions = query_openai('text-davinci-003', prompt, n_candidates*max_highlight_len*2, 0.9)

In [77]:
questions

'\n\nDoes the city have a National Park?\nDoes the city have a Memorial or Monument?\nDoes the city have a Lake or River?\nDoes the city have any Castles or other Historical Sites?\nDoes the city have any Theatres or Museums?\nDoes the city have any Parks or Gardens?'

### Based on the touristic highlights, populate the answer for each question and city pair

In [91]:
answers = {}

for question in questions.strip().split('\n'):
    if question not in answers:
        answers[question] = {}
    
    for candidate in all_candidates:
        prompt = f"Based on the city characteristics below, answer the question for a given city.\nCity characteristics:{highlights}\nQuestion: {question}\nCity: {candidate}\nAnswer:"
        answer = query_openai('text-davinci-003', prompt, 1, 1.0)
        if answer not in answers[question]:
            answers[question][answer] = []
        answers[question][answer].append(candidate)
    
    print(question)
    for answer in answers[question]:
        print(f"{answer}: {len(answers[question][answer])}")
    print('*'*50)

Does the city have a National Park?
 Yes: 12
 No: 9
**************************************************
Does the city have a Memorial or Monument?
 Yes: 21
**************************************************
Does the city have a Lake or River?
 Yes: 1
 No: 20
**************************************************
Does the city have any Castles or other Historical Sites?
 No: 2
 Yes: 19
**************************************************
Does the city have any Theatres or Museums?
 Yes: 21
**************************************************
Does the city have any Parks or Gardens?
 Yes: 21
**************************************************


### Save for future use

In [100]:
city_q_and_a = {city : {} for city in all_candidates}

for question in answers:
    short_q = ' '.join(question.split()[5:])
    for answer in answers[question]:
        for city in answers[question][answer]:
            city_q_and_a[city][short_q] = int('Yes' in answer)

q_and_a_tuples = []

for city in city_q_and_a:
    q_and_a_tuple = {'city' : city}
    for short_q in city_q_and_a[city]:
        q_and_a_tuple[short_q] = city_q_and_a[city][short_q]
    q_and_a_tuples.append(q_and_a_tuple)

df_q_and_a = pd.DataFrame(q_and_a_tuples)
df_q_and_a.head()

,city,National Park?,Memorial or Monument?,Lake or River?,Castles or other Historical Sites?,Theatres or Museums?,Parks or Gardens?
0,"Brasília, Brazil",1,1,1,0,1,1
1,"Quito, Ecuador",0,1,0,1,1,1
2,"San Jose, Costa Rica",1,1,0,1,1,1
3,"La Paz, Bolivia",0,1,0,0,1,1
4,"Asuncion, Paraguay",0,1,0,1,1,1


In [101]:
df_q_and_a.to_csv('q_and_a_example.csv')

In [102]:
! ls -lah *.csv

-rw-r--r--  1 vbursztyn  staff    32K May  3 23:28 candidates.csv
-rw-r--r--  1 vbursztyn  staff   908B May  4 19:25 q_and_a_example.csv
